In [ ]:
import pandas as pd
import pandas_utils as pu

import matplotlib.pyplot as plt
import matplotlib
import upsetplot as ups
import seaborn as sns

In [ ]:
library = "all"
def save_fig(file_name):
  plt.rcParams["pdf.fonttype"] = 42
  plt.savefig(r"C:\git\msn_library\figures/{}/{}.png".format(library, file_name), dpi=300, bbox_inches="tight", transparent=True)
  plt.savefig(r"C:\git\msn_library\figures/{}/{}.pdf".format(library, file_name), bbox_inches="tight", transparent=True)
  plt.savefig(r"C:\git\msn_library\figures/{}/{}.svg".format(library, file_name), bbox_inches="tight", transparent=True)
sns.set_theme(font_scale=1, style="white")

## Overlap between public databases

In [ ]:
libraries = {
  "acquired": r"C:\git\msn_library\data\acquisition_results\all_lib_only_detected.tsv",
  "massbankeu":  r"C:\git\msn_library\data\public_library\20231208\MassBank_nist.tsv",
  "mona": r"C:\git\msn_library\data\public_library\20231208\MoNA_LC_MSMS.tsv",
  "gnps": r"C:\git\msn_library\data\gnpslib\20240229_ALL_GNPS_NO_PROPOGATED_standardized.tsv",
  "mzcloud": r"C:\git\msn_library\data\public_library\mzCloud_04.03.2024_for Customer_standardized.tsv",
  "nist20": r"C:\git\msn_library\data\public_library\NIST20_standardized.tsv"
}

# acquired = [
#   "mce_bioactive",
#   "nih_natural_product",
#   "mce_scaffold_library",
#   "iocb_peptide",
# ]

open = [
  "massbankeu",
  "mona",
  "gnps",
]

commercial = [
  "mzcloud",
  "nist20"
]


In [ ]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value)[["split_inchikey"]].copy()
  df = df.dropna(subset="split_inchikey").drop_duplicates(["split_inchikey"]).set_index(["split_inchikey"])
  df[key] = True
  # if key in acquired:
  #   df["acquired"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)


merged_df = pd.concat(dfs, axis=1)
merged_df["entries"] = merged_df.count(axis=1)
# merged_df["split_inchikey"] = [rdkit.split_inchikey(inchikey) for inchikey in merged_df.index]
# merged_df["acquired"] = merged_df[acquired].any(axis=1)
merged_df["open"] = merged_df[open].any(axis=1)
merged_df["commercial"] = merged_df[commercial].any(axis=1)
merged_df["available"] = merged_df[open].any(axis=1) | merged_df[commercial].any(axis=1)


merged_df

In [ ]:
# pu.save_dataframe(merged_df, r"C:\git\msn_library\data\acquisition_results\public_spectral_libraries_and_new.tsv")

In [ ]:
merged_df[(merged_df["open"] == True)]

In [ ]:
merged_df[(merged_df["acquired"] == True) &  (merged_df["gnps"] == True)]

In [ ]:
merged_df[(merged_df["acquired"] == True) &  (merged_df["commercial"] == False)]

In [ ]:
merged_df[(merged_df["acquired"] == True) &  (merged_df["available"] == False)]

In [ ]:
merged_df[merged_df["mzcloud"] == True]

In [ ]:
acquired_df = merged_df[(merged_df["acquired"] == True)]
acquired_df

In [ ]:
file_name = "acquired_combined_public_separated"
acquired_combined = ups.from_indicators(["acquired", "massbankeu", "mona", "gnps", "nist20", "mzcloud"], data=merged_df)
ups.plot(acquired_combined)
save_fig(file_name)
matplotlib.pyplot.show()

In [ ]:
file_name = "acquired_combined_open_combined"
acquired_combined = ups.from_indicators(["acquired", "open", "nist20", "mzcloud"], data=merged_df)
ups.plot(acquired_combined)
save_fig(file_name)
matplotlib.pyplot.show()

In [ ]:
open = merged_df[merged_df["open"] == True]
file_name = "open_libraries_upsetplot"
acquired_combined = ups.from_indicators(["massbankeu", "mona", "gnps"], data=open)
ups.plot(acquired_combined)
save_fig(file_name)

In [ ]:
open

In [ ]:
new_open = merged_df[(merged_df["open"] == True) | (merged_df["acquired"] == True)]
file_name = "new_and_open_libraries_upsetplot"
acquired_combined = ups.from_indicators(["acquired", "massbankeu", "mona", "gnps"], data=new_open)
ups.plot(acquired_combined)
save_fig(file_name)


In [ ]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value)[["canonical_smiles"]].copy()
  df = df.dropna(subset="canonical_smiles").drop_duplicates(["canonical_smiles"]).set_index(["canonical_smiles"])
  df[key] = True
  # if key in acquired:
  #   df["acquired"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)


merged_df = pd.concat(dfs, axis=1)
merged_df["entries"] = merged_df.count(axis=1)
# merged_df["split_inchikey"] = [rdkit.split_inchikey(inchikey) for inchikey in merged_df.index]
# merged_df["acquired"] = merged_df[acquired].any(axis=1)
merged_df["open"] = merged_df[open].any(axis=1)
merged_df["commercial"] = merged_df[commercial].any(axis=1)
merged_df["available"] = merged_df[open].any(axis=1) | merged_df[commercial].any(axis=1)


merged_df

In [ ]:
file_name = "acquired_combined_open_combined_canonical_smiles"
acquired_combined = ups.from_indicators(["acquired", "open", "nist20", "mzcloud"], data=merged_df)
ups.plot(acquired_combined)
save_fig(file_name)
matplotlib.pyplot.show()